In [2]:
# Gerekli kütüphaneleri içe aktarıyoruz
import pandas as pd      # Veri analizi için
import numpy as np       # Sayısal işlemler için
import matplotlib.pyplot as plt  # Grafik çizimi için
import seaborn as sns    # İstatistiksel görselleştirme için

# Grafiklerin notebook içinde görünmesi için
%matplotlib inline

# Uyarıları kapatmak için (isteğe bağlı)
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Pandas ile veri setini yüklüyoruz
df = pd.read_csv('marketing_campaign.csv', sep='\t')  # Eğer dosya virgül ile ayrılmışsa sep=',' yap

# İlk 5 satırı görüntüleyerek kontrol ediyoruz
df.head()

In [ ]:
# Sütun isimleri ve veri tiplerini listeliyoruz
print("Sütun isimleri ve veri tipleri:")
print(df.dtypes)

In [ ]:
# Her sütunda kaç adet eksik değer olduğunu kontrol ediyoruz
print("Eksik değer sayısı:")
print(df.isnull().sum())

In [ ]:
# Sayısal sütunlar için temel istatistikleri görüntülüyoruz
df.describe().T

In [ ]:
# Kategorik sütunların benzersiz değerlerini ve dağılımlarını inceliyoruz
kategorik_sutunlar = ['Education', 'Marital_Status']
for sutun in kategorik_sutunlar:
    print(f"{sutun} sütununda bulunan değerler ve adetleri:")
    print(df[sutun].value_counts())
    print("-" * 30)

In [ ]:
# Yaş dağılımı için histogram çiziyoruz
plt.figure(figsize=(8,4))
sns.histplot(2024 - df['Year_Birth'], bins=30, kde=True)
plt.title('Müşteri Yaş Dağılımı')
plt.xlabel('Yaş')
plt.ylabel('Müşteri Sayısı')
plt.show()

In [ ]:
# Gelir dağılımı için histogram çiziyoruz
plt.figure(figsize=(8,4))
sns.histplot(df['Income'].dropna(), bins=30, kde=True)
plt.title('Gelir Dağılımı')
plt.xlabel('Gelir')
plt.ylabel('Müşteri Sayısı')
plt.show()

In [ ]:
# 'Income' sütununda eksik değeri olan satırları siliyoruz
df = df.dropna(subset=['Income'])

# Kalan eksik değerleri tekrar kontrol ediyoruz
print("Eksik değer sayısı (güncel):")
print(df.isnull().sum())

In [ ]:
# Yaş sütunu oluşturuyoruz (2024 yılına göre)
df['Age'] = 2024 - df['Year_Birth']

# Yaş dağılımını tekrar kontrol ediyoruz
print(df['Age'].describe())

# Yaş için uç değerleri temizliyoruz (18-100 arası)
df = df[(df['Age'] >= 18) & (df['Age'] <= 100)]

# Temizlenmiş yaş dağılımını tekrar kontrol ediyoruz
print(df['Age'].describe())

In [ ]:
# Gelir sütununun üst sınırını belirliyoruz (99. persentil)
income_upper = df['Income'].quantile(0.99)

# 99. persentil üzerindeki gelirleri analiz dışında bırakıyoruz
df = df[df['Income'] <= income_upper]

# Temizlenmiş gelir dağılımını tekrar kontrol ediyoruz
print(df['Income'].describe())

In [ ]:
# Marital_Status sütununda nadir ve anlamsız kategorileri 'Other' olarak gruplayalım
df['Marital_Status'] = df['Marital_Status'].replace(
    ['Alone', 'Absurd', 'YOLO'], 'Other'
)

# Son dağılımı kontrol edelim
print(df['Marital_Status'].value_counts())

In [ ]:
# Son veri setinin boyutunu ve ilk 5 satırını tekrar kontrol ediyoruz
print("Veri setinin son hali (satır, sütun):", df.shape)
df.head()


In [ ]:
# Kategorik değişkenleri one-hot encoding ile sayısallaştırıyoruz
df_encoded = pd.get_dummies(df, columns=['Education', 'Marital_Status'], drop_first=True)

# Sonuçları kontrol ediyoruz
print("Yeni sütunlar:", df_encoded.columns)
df_encoded.head()

In [ ]:
from sklearn.preprocessing import StandardScaler

# Segmentasyon için kullanılacak sütunları seçiyoruz
# ID, Dt_Customer, Year_Birth, Z_CostContact, Z_Revenue gibi analizde kullanılmayacak sütunları çıkarıyoruz
drop_cols = ['ID', 'Year_Birth', 'Dt_Customer', 'Z_CostContact', 'Z_Revenue']
X = df_encoded.drop(columns=drop_cols)

# Ölçeklendirme işlemi
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Ölçeklendirilmiş veriyi DataFrame olarak saklayalım
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# İlk 5 satırı kontrol edelim
X_scaled_df.head()

In [ ]:
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt

# Farklı küme sayıları için K-Means algoritmasını çalıştırıp, WCSS değerlerini (küme içi toplam kareler) hesaplıyoruz
wcss = []
K = range(2, 21)  # 2'den 10'a kadar küme sayısını deniyoruz

for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_scaled_df)
    wcss.append(kmeans.inertia_)  # inertia_ = WCSS

# Sonuçları çiziyoruz
plt.figure(figsize=(8,4))
plt.plot(K, wcss, 'bo-')
plt.xlabel('Küme Sayısı (k)')
plt.ylabel('WCSS (Küme İçi Toplam Kareler)')
plt.title('Elbow (Dirsek) Yöntemi ile Optimum Küme Sayısı')
plt.show()

In [ ]:
from sklearn.metrics import silhouette_score

# Farklı küme sayıları için silhouette skorunu hesaplıyoruz
silhouette_scores = []
for k in K:
    kmeans = KMeans(n_clusters=k, random_state=42)
    labels = kmeans.fit_predict(X_scaled_df)
    score = silhouette_score(X_scaled_df, labels)
    silhouette_scores.append(score)

# Sonuçları çiziyoruz
plt.figure(figsize=(8,4))
plt.plot(K, silhouette_scores, 'ro-')
plt.xlabel('Küme Sayısı (k)')
plt.ylabel('Ortalama Silhouette Skoru')
plt.title('Silhouette Yöntemi ile Optimum Küme Sayısı')
plt.show()

In [ ]:
# Optimum küme sayısını seçiyoruz (örneğin k=4)
k_opt = 4  # Grafiğe göre değiştir

# K-Means modelini eğitiyoruz
kmeans = KMeans(n_clusters=k_opt, random_state=42)
clusters = kmeans.fit_predict(X_scaled_df)

# Sonuçları orijinal veri setine ekliyoruz
df['Segment'] = clusters

# Her segmentte kaç müşteri olduğunu kontrol ediyoruz
print(df['Segment'].value_counts())

In [ ]:
# Her segmentin temel istatistiklerini görüntülüyoruz
segment_analiz = df.groupby('Segment').mean(numeric_only=True)
print(segment_analiz)

# Segmentlerin yaş ve gelir dağılımını görselleştiriyoruz
plt.figure(figsize=(8,6))
sns.boxplot(x='Segment', y='Age', data=df)
plt.title('Segmentlere Göre Yaş Dağılımı')
plt.show()

plt.figure(figsize=(8,6))
sns.boxplot(x='Segment', y='Income', data=df)
plt.title('Segmentlere Göre Gelir Dağılımı')
plt.show()

In [ ]:
# Segmentlere göre harcama ortalamalarını hesaplıyoruz
harcama_kolonlari = [
    'MntWines', 'MntFruits', 'MntMeatProducts',
    'MntFishProducts', 'MntSweetProducts', 'MntGoldProds'
]

segment_harcama = df.groupby('Segment')[harcama_kolonlari].mean().T

# Sonuçları tablo olarak yazdırıyoruz
print(segment_harcama)

# Her segmentin harcama profilini görselleştiriyoruz
plt.figure(figsize=(12,6))
segment_harcama.plot(kind='bar')
plt.title('Segmentlere Göre Ortalama Harcamalar')
plt.ylabel('Ortalama Harcama (Son 2 Yıl)')
plt.xlabel('Ürün Türü')
plt.legend(title='Segment')
plt.tight_layout()
plt.show()

In [ ]:
kampanya_kolonlari = [
    'AcceptedCmp1', 'AcceptedCmp2', 'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'Response'
]

segment_kampanya = df.groupby('Segment')[kampanya_kolonlari].mean().T

print(segment_kampanya)

# Görselleştirme
plt.figure(figsize=(12,6))
segment_kampanya.plot(kind='bar')
plt.title('Segmentlere Göre Kampanya Yanıt Oranları')
plt.ylabel('Ortalama Yanıt Oranı')
plt.xlabel('Kampanya')
plt.legend(title='Segment')
plt.tight_layout()
plt.show()

In [ ]:
kanal_kolonlari = [
    'NumWebPurchases', 'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth'
]

segment_kanal = df.groupby('Segment')[kanal_kolonlari].mean().T

print(segment_kanal)

# Görselleştirme
plt.figure(figsize=(12,6))
segment_kanal.plot(kind='bar')
plt.title('Segmentlere Göre Alışveriş Kanalı Tercihleri')
plt.ylabel('Ortalama Alışveriş Sayısı')
plt.xlabel('Kanal')
plt.legend(title='Segment')
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10,6))
sns.scatterplot(x='Age', y='Income', hue='Segment', data=df, palette='tab10')
plt.title('Segmentlerin Yaş ve Gelir Ekseni Üzerinde Dağılımı')
plt.xlabel('Yaş')
plt.ylabel('Gelir')
plt.legend(title='Segment')
plt.show()